In [996]:
!pip3 install statsmodels
!pip3 install vaderSentiment
!pip3 install nbconvert
!pip3 install tabulate
!pip3 install --upgrade scipy
!pip3 install nltk


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 3.2 MB/s e

     |████████████████████████████████| 756 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 6.9 MB/s  eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [164]:
import numpy as np
import pandas as pd

from IPython.display import display
from statistics import mean

from datetime import date, timedelta
import datetime

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.gof import chisquare as chisquare
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from statsmodels.stats.anova import anova_lm
from statsmodels.tools.tools import pinv_extended

import csv
from patsy import ModelDesc
from tabulate import tabulate

import scipy


In [352]:
current_date = datetime.datetime.strptime("2020-08-22", "%Y-%m-%d").date()

def update_end_date(row, columnName, latestDate):
    if row[columnName] == row[columnName]:
        return row[columnName]
    else:
        return latestDate

def getDays(row, beginColumnName, endColumnName):
    return (row[endColumnName] - row[beginColumnName]).days

def calculate_recency(row, columnName):
    return (current_date - row[columnName]).days

def lookup_index(row, columnName, array):
    if(row[columnName] not in array):
        return -1
    return array.index(row[columnName]) + 1

def fix_signficance(row):
    if('significance' in row['valence']):
        return row['valence']
    else:
        return row['significance']

def fix_valence(row):
    if('significance' in row['significance']):
        return row['valence']
    else:
        return row['significance']  

def compute_sentiment(row):
    post = row['Text']
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(post)
    sentiment = 0
    if (vs["neu"]>0.8):
        sentiment = 0
    elif (vs["pos"]==vs["neg"]):
        sentiment = 0
    elif (vs["pos"]>vs["neg"]):
        sentiment = 1
    elif (vs["neg"]>vs["neu"]):
        sentiment = -1
    return sentiment

def convert_valence_to_sentiment(row):
    valence = row['valence']
    retVal = 0
    if (valence == 'Neither Positive or Negative'):
        retVal = 0
    elif("Positive" in valence):
        retVal = 1
    elif("Negative" in valence):
        retVal = -1
    return retVal

def update_status(row, columnName):
    if row[columnName] == row[columnName]:
        if "ongoing" in row[columnName].lower():
            return "Ongoing"
        return row[columnName]
    else:
        return "Ended"

def update_education_level(row, columnName):
    if 'college' in row[columnName].lower():
        return "College"
    elif 'doctoral' in row[columnName].lower():
        return "Doctoral"
    elif 'master' in row[columnName].lower() or 'grad' in row[columnName].lower():
        return "Graduate"
    elif 'hs' in row[columnName].lower() or 'high school' in row[columnName].lower():
        return "High"
    
    return row[columnName]

def normalize(series):
    return (series - series.mean())/series.std()

def star_pvals(p_val):
    if p_val<0.001:
        return "***"
    elif p_val<0.01:
        return "**"
    elif p_val<0.05:
        return "*"
    return "."

def no_star_pvals(p_val):
    if p_val<0.001:
        return 3
    elif p_val<0.01:
        return 2
    elif p_val<0.05:
        return 1
    return 0

In [3]:
control_variables             = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender'
combined_control_variables    = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + data_type'

sr_life_event_variables       = 'Anticipation_sr + LifeEventType_sr + valence_sr + recency_sr + status_sr + Intimacy_sr + Scope_sr + significance_sr'
sm_life_event_variables       = 'Anticipation_sm + LifeEventType_sm + valence_sm + recency_sm + status_sm + Intimacy_sm + Scope_sm + significance_sm'
combined_life_event_variables = 'Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'

sr_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + significance_label'
sm_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope'
combined_all_variables        = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'
# combined_sm_sr_all_variables  = 'shipley_vocab_sm + shipley_vocab_sr + shipley_abs_sm + shipley_abs_sr + openness_sm + openness_sr + conscientiousness_sm + conscientiousness_sr + extraversion_sm + extraversion_sr + agreeableness_sm + agreeableness_sr + neuroticism_sm + neuroticism_sr + pos_affect_sm + pos_affect_sr + neg_affect_sm + neg_affect_sr + stai_trait_sm + stai_trait_sr + education_level_sm + education_level_sr + psqi_sm + psqi_sr + age_sm + age_sr + gender_sm + gender_sr + Anticipation_sm + Anticipation_sr + LifeEventFamily_sm + LifeEventFamily_sr + valence_sm + valence_sr + Intimacy_sm + Intimacy_sr + Scope_sm + Scope_sr + data_type'

In [5]:
def load_demographics_data():
    demographics_data = pd.read_csv('data/igtbs_demographics_complete.csv', parse_dates=True)
    demographics_data = demographics_data[['age','gender','snapshot_id', 'shipley.vocab', 'shipley.abs', 'openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism','pos.affect','neg.affect','stai.trait','psqi','educ']]
    demographics_data['education_level'] = demographics_data.apply(update_education_level, columnName='educ', axis=1)
    demographics_data = demographics_data.drop(columns=['educ'])
    demographics_data = demographics_data.rename(columns={
        'shipley.vocab': 'shipley_vocab',
        'shipley.abs': 'shipley_abs',
        'pos.affect': 'pos_affect',
        'neg.affect': 'neg_affect',
        'stai.trait': 'stai_trait'
    })
    demographics_data = demographics_data.dropna()
    return demographics_data

In [6]:
def load_survey_categories():
    df_self_reported_categories = pd.read_csv('data/Life Events Categories Mapping - Self-Reported Categories.csv')
    return df_self_reported_categories

def load_survey_data_without_categories():
    df_survey = pd.read_csv('data/Superimposed/LifeEvents_Curated_non_blinded.csv', parse_dates=True)    
    df_survey = df_survey[['snapshot_id', 'description','UpdatedBeginDate', 'UpdatedEndDate','life_event_type', 'work_perf_impact', 'significance','valence', 'ended_or_ongoing']]

    # Date manipulation
    latest_date = max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedEndDate'])
    latest_date = max(latest_date, max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedBeginDate']))
    df_survey = df_survey.drop(df_survey[df_survey['UpdatedBeginDate'].isnull() == True].index)

    df_survey['UpdatedBeginDate'] = pd.to_datetime(df_survey['UpdatedBeginDate'], format = '%Y-%m-%d').dt.date
    df_survey['UpdatedEndDate'] = df_survey.apply(update_end_date, columnName='UpdatedEndDate', latestDate=latest_date, axis=1)
    df_survey['UpdatedEndDate'] = pd.to_datetime(df_survey['UpdatedEndDate'], format = '%Y-%m-%d').dt.date

    df_survey['num_of_days'] = df_survey.apply(getDays, endColumnName='UpdatedEndDate', beginColumnName='UpdatedBeginDate', axis=1)
    df_survey['recency'] = df_survey.apply(calculate_recency, columnName='UpdatedEndDate', axis=1)

    # Update values for valence and significance
    df_survey.replace({'valence': {np.nan: 'Neither Positive or Negative'}, 'significance': {np.nan: 'Neither Positive or Negative'}}, inplace=True)
    df_survey['fixed_signficance'] = df_survey.apply(fix_signficance, axis = 1)
    df_survey['fixed_valence'] = df_survey.apply(fix_valence, axis = 1)
    df_survey = df_survey.drop(columns = ['valence', 'significance'])
    df_survey = df_survey.rename(columns={"fixed_signficance": "significance", "fixed_valence": "valence"})
    df_survey['valence'] = df_survey.apply(convert_valence_to_sentiment, axis=1)
    df_survey['ended_or_ongoing'] = df_survey.apply(update_status, columnName='ended_or_ongoing', axis=1)

    # Select columns we are interested in
    df_survey = df_survey[['snapshot_id', 'description', 'UpdatedBeginDate', 'UpdatedEndDate', 'significance', 'valence', 'ended_or_ongoing', 'recency']]

    # Label encoding for significance
    le_significance = LabelEncoder()
    le_significance.fit(df_survey['significance'].values)
    df_survey['significance_label'] = df_survey.apply(lambda x: le_significance.transform([x['significance']])[0], axis=1)
    df_survey = df_survey.drop(columns=['significance'])

    return df_survey

def load_survey_data():
    df_survey_without_categories = load_survey_data_without_categories()
    df_self_reported_categories = load_survey_categories()
    df_survey = pd.merge(df_survey_without_categories, df_self_reported_categories, how="inner", left_on="description", right_on="SR_LifeEvent")
    df_survey = df_survey.drop(columns=['description', 'SR_LifeEvent', 'LifeEventFinal', 'LifeEventFamily2'])
    df_survey['significance_label'] = (df_survey['significance_label'] - df_survey['significance_label'].min()) / (df_survey['significance_label'].max() - df_survey['significance_label'].min())    
    df_survey = df_survey.rename(columns={'ended_or_ongoing':'status', 'LifeEventFamily': 'LifeEventType', 'significance_label': 'significance'})


    return df_survey

In [15]:
sm = load_social_media_data()

In [21]:
s = sm.groupby(['snapshot_id', 'created_date']).count().reset_index()
s['status'].value_counts()

1     1422
2      160
3       48
4       12
5        6
10       1
6        1
Name: status, dtype: int64

In [7]:
def load_social_media_categories():
    df_social_media_categories = pd.read_csv('data/Life Events Categories Mapping - Social Media Categories-2.csv')
    df_social_media_categories['SignificanceRank'] = df_social_media_categories['SignificanceRank'].fillna(df_social_media_categories['SignificanceRank'].max())
    return df_social_media_categories

def load_social_media_data_without_categories():
    df_social_media_data = pd.read_csv('data/Superimposed/Facebook Data For Life Events-Combined - FB Data.csv')
    df_social_media_data = df_social_media_data[['snapshot_id', 'created_time', 'Text', 'final_life_event_category_2', 'ended/ongoing']]
    df_social_media_data = df_social_media_data.replace({'PostiveMove':'PositiveMove', 'Negative Move':'NegativeMove'})
    df_social_media_data = df_social_media_data.drop(df_social_media_data[((df_social_media_data['final_life_event_category_2'].isnull() == True))].index)
    df_social_media_data['created_date'] = pd.to_datetime(df_social_media_data['created_time'], format = '%Y-%m-%d %H:%M:%S').dt.date
    df_social_media_data['valence'] = df_social_media_data.apply(compute_sentiment, axis=1)
    df_social_media_data = df_social_media_data.drop(columns=['created_time','Text'])
    df_social_media_data['recency'] = df_social_media_data.apply(calculate_recency, columnName='created_date', axis=1)
    df_social_media_data['ended/ongoing'] = df_social_media_data.apply(update_status, columnName='ended/ongoing', axis=1)
    return df_social_media_data

def load_social_media_data():
    df_social_media_data = load_social_media_data_without_categories()
    df_social_media_categories = load_social_media_categories()
    df_social_media_data_with_categories = pd.merge(df_social_media_data, df_social_media_categories, how="inner", left_on='final_life_event_category_2', right_on='SM_LifeEvent')
    df_social_media_data_with_categories = df_social_media_data_with_categories.drop(columns=['final_life_event_category_2','SM_LifeEvent','LifeEventFamily2','Comments'])
    df_social_media_data_with_categories = df_social_media_data_with_categories.rename(columns={'ended/ongoing':'status', 'LifeEventFamily': 'LifeEventType', 'SignificanceRank': 'significance'})
    df_social_media_data_with_categories['significance'].fillna(df_social_media_data_with_categories['significance'].max())
    df_social_media_data_with_categories['significance'] = (df_social_media_data_with_categories['significance'] - df_social_media_data_with_categories['significance'].min()) / (df_social_media_data_with_categories['significance'].max() - df_social_media_data_with_categories['significance'].min())    
    
    return df_social_media_data_with_categories

In [8]:
def load_dailies_data():
    df_dailies = pd.read_csv('data/Superimposed/dailies_scores.csv', low_memory=False)
    df_dailies = df_dailies[['snapshot_id','day', 'alc_status', 'alc.quantity.d', 'anxiety.d', 'pos.affect.d', 'neg.affect.d','sleep.d', 'stress.d']]
    df_dailies['day'] = pd.to_datetime(df_dailies['day'], format='%Y-%m-%d').dt.date
    df_dailies = df_dailies.rename(columns={'alc.quantity.d': 'alc_quantity',
    'anxiety.d': 'anxiety',
    'pos.affect.d': 'pos_affect',
    'neg.affect.d': 'neg_affect',
    'sleep.d': 'sleep',
    'stress.d': 'stress'})
    df_dailies['sleep'] = df_dailies['sleep'] + 1
    return df_dailies

In [9]:
def calculate_year(row, columnName):
    return row[columnName].isocalendar()[0]

def calculate_week(row, columnName):
    return row[columnName].isocalendar()[1]

In [85]:
load_social_media_data()

,snapshot_id,status,created_date,valence,recency,LifeEventType,Anticipation,Intimacy,Scope,significance
0,12840161237957324034,Ongoing,2018-03-16,0,890,Personal,Anticipated,2,2,0.794118
1,11156103277197680506,Ongoing,2018-07-31,1,753,Personal,Anticipated,2,2,0.794118
2,11329586216091316598,Ongoing,2018-09-08,0,714,Personal,Anticipated,2,2,0.794118
3,13543308681200334026,Ongoing,2018-07-22,0,762,Personal,Anticipated,2,2,0.794118
4,12676581714475000627,Ongoing,2018-04-25,0,850,Personal,Anticipated,2,2,0.794118
...,...,...,...,...,...,...,...,...,...,...
1975,10729524556661174446,Ongoing,2018-12-19,0,612,School,Unanticipated,3,2,0.833333
1976,9767765376628075157,Ongoing,2018-07-27,0,757,Health,Anticipated,3,3,0.465686
1977,9767765376628075157,Ongoing,2018-10-23,0,669,Health,Anticipated,3,3,0.465686
1978,9767765376628075157,ended,2018-09-19,1,703,Health,Unanticipated,3,3,0.176471


In [83]:
def weekly_avg(row, data):
    days_3 = timedelta(days=3)
    days_1 = timedelta(days=1)
    before = row['day'] - days_3
    after = row['day'] + days_3
    
    
def calculate_avg_mental_health():
    df_dailies = load_dailies_data()
    v = load_dailies_data()[['snapshot_id', 'day', 'stress', 'sleep', 'anxiety']].values
    data = {}
    for i in v:
        data[(i[0], i[1])] = (i[2], i[3], i[4])
    final_data = []
    
    for user in df_dailies['snapshot_id'].unique():
        min_date = df_dailies[df_dailies['snapshot_id'] == user]['day'].min()
        max_date = df_dailies[df_dailies['snapshot_id'] == user]['day'].max()
        days_1 = timedelta(days=1)
        days_3 = timedelta(days=3)

        while(min_date < max_date):
            date = min_date
            before = date - days_3
            after = date + days_3
        
            accumulate_stress = []
            accumulate_sleep = []
            accumulate_anxiety = []

            while before < after:
                if (user, before) in data:
                    cache = data[(user, before)]
                    if cache[0] == cache[0]:
                        accumulate_stress.append(cache[0])
                    if cache[1] == cache[1]:
                        accumulate_sleep.append(cache[1])
                    if cache[2] == cache[2]:
                        accumulate_anxiety.append(cache[2])
                before+=days_1

            if len(accumulate_stress) > 0:
                avg_stress = sum(accumulate_stress) / len(accumulate_stress)
            else:
                avg_stress = -1
            if len(accumulate_sleep) > 0:
                avg_sleep = sum(accumulate_sleep) / len(accumulate_sleep)
            else:
                avg_sleep = -1
            if len(accumulate_anxiety) > 0:
                avg_anxiety = sum(accumulate_anxiety) / len(accumulate_anxiety)
            else:
                avg_anxiety = -1

            final_data.append([user, min_date, avg_stress, avg_sleep, avg_anxiety])
            min_date+=days_1
    final_data = pd.DataFrame(final_data, columns=['snapshot_id', 'day', 'stress', 'sleep', 'anxiety'])
    return final_data[(final_data['stress'] != -1) & (final_data['sleep'] != -1) & (final_data['anxiety'] != -1)]


In [84]:
calculate_avg_mental_health()

,snapshot_id,day,stress,sleep,anxiety
0,10003528337325499062,2018-02-28,1.666667,8.000000,2.000000
1,10003528337325499062,2018-03-01,1.750000,8.000000,1.750000
2,10003528337325499062,2018-03-02,1.600000,8.666667,1.600000
3,10003528337325499062,2018-03-03,1.500000,8.666667,1.500000
4,10003528337325499062,2018-03-04,1.500000,8.750000,1.333333
...,...,...,...,...,...
47115,9999689421615292586,2018-06-27,1.600000,8.666667,1.200000
47116,9999689421615292586,2018-06-28,1.600000,8.500000,1.200000
47117,9999689421615292586,2018-06-29,1.600000,8.500000,1.200000
47118,9999689421615292586,2018-06-30,1.600000,8.000000,1.200000


In [140]:
def build_stacked_df(dependent_variable):
    try:
        merged_data = pd.read_csv('Linear Regression/Relative Week/all_data_stacked_'+dependent_variable+'.csv')
    except:
        df_dailies = calculate_avg_mental_health()
        df_social_media = load_social_media_data()
        df_survey = load_survey_data()
        df_demographics = load_demographics_data()

        df_social_media['type'] = 'Social Media'
        df_social_media['day'] = df_social_media['created_date']
        
        df_survey['day'] = -1
        survey_rows = []
#         print(df_survey.columns)
        for i, row in df_survey.iterrows():
            s_date = row['UpdatedBeginDate']
            e_date = row['UpdatedEndDate']
            delta = timedelta(days=1)

            while s_date <= e_date:
                r = row.copy(deep=True)
                r['day'] = s_date
                r['recency'] = (current_date - s_date).days
                survey_rows.append(r.values)
                s_date +=delta
#         print(len(survey_rows[0]))
#         print(df_survey.columns)
#         print(df_social_media.columns)
        df_survey = pd.DataFrame(survey_rows, columns=df_survey.columns)
        df_survey['type'] = 'Survey'

        df_dailies = df_dailies[['snapshot_id', dependent_variable, 'day']]

        merged_data = pd.merge(df_dailies, df_demographics, how="inner", on="snapshot_id")
        merged_data_sm = pd.merge(merged_data, df_social_media, how="inner", on=["snapshot_id", "day"])
        merged_data_sr = pd.merge(merged_data, df_survey, how="inner", on=["snapshot_id", "day"])

        merged_data = pd.concat([merged_data_sm,merged_data_sr],ignore_index=True)
        
        merged_data['recency'] = (merged_data['recency'] - merged_data['recency'].min()) / (merged_data['recency'].max() - merged_data['recency'].min())    
        merged_data.to_csv('Linear Regression/Relative Week/all_data_stacked_'+dependent_variable+'.csv', index=False)
        
    merged_data = merged_data.drop(columns=['created_date', 'UpdatedBeginDate', 'UpdatedEndDate'])
    return merged_data

In [123]:
def build_merged_df(dependent_variable):
    try:
        merged_data = pd.read_csv('Linear Regression/Relative Week/all_data_'+dependent_variable+'.csv')
    except:
        df_dailies = calculate_avg_mental_health()
        df_social_media = load_social_media_data()
        df_survey = load_survey_data()
        df_demographics = load_demographics_data()
        df_social_media.rename(columns=lambda x: x+'_sm', inplace=True)
        df_social_media = df_social_media.rename(columns={
            'snapshot_id_sm': 'snapshot_id',
            'created_date_sm': 'day'
        })
        df_social_media['recency_sm'] = (df_social_media['recency_sm'] - df_social_media['recency_sm'].min()) / (df_social_media['recency_sm'].max() - df_social_media['recency_sm'].min())
        df_survey['day'] = -1
        survey_rows = []
        for i, row in df_survey.iterrows():
            s_date = row['UpdatedBeginDate']
            e_date = row['UpdatedEndDate']
            delta = timedelta(days=1)

            while s_date <= e_date:
                r = row.copy(deep=True)
                r['day'] = s_date
                r['recency'] = (current_date - s_date).days
                survey_rows.append(r.values)
                s_date +=delta

        df_survey = pd.DataFrame(survey_rows, columns=df_survey.columns)
        df_survey.rename(columns=lambda x: x+'_sr', inplace=True)
        df_survey = df_survey.rename(columns={
            'snapshot_id_sr': 'snapshot_id',
            'day_sr' : 'day'
        })

        df_dailies = df_dailies[['snapshot_id', dependent_variable, 'day']]
        df_survey['recency_sr'] = (df_survey['recency_sr'] - df_survey['recency_sr'].min()) / (df_survey['recency_sr'].max() - df_survey['recency_sr'].min())    

        merged_data = pd.merge(df_dailies, df_demographics, how="inner", on="snapshot_id")
        merged_data = pd.merge(merged_data, df_social_media, how="left", on=["snapshot_id", "day"])
        merged_data = pd.merge(merged_data, df_survey, how="left", on=["snapshot_id", "day"])

        merged_data.to_csv('Linear Regression/Relative Week/all_data_'+dependent_variable+'.csv', index=False)
        
#     merged_data = merged_data.drop(columns=['created_date_sm', 'UpdatedBeginDate_sr', 'UpdatedEndDate_sr'])
    return merged_data


## Run Regression  

In [310]:
control_variables = ['shipley_vocab', 'shipley_abs', 'openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism', 'pos_affect', 'neg_affect', 'stai_trait', 'education_level', 'psqi', 'age', 'gender']
sr_life_event_variables = ['Anticipation_sr', 'LifeEventType_sr', 'valence_sr', 'recency_sr', 'status_sr', 'Intimacy_sr', 'Scope_sr', 'significance_sr']
sm_life_event_variables = ['Anticipation_sm', 'LifeEventType_sm', 'valence_sm', 'recency_sm', 'status_sm', 'Intimacy_sm', 'Scope_sm', 'significance_sm']
life_event_variables = ['Anticipation', 'LifeEventType', 'valence', 'recency', 'status', 'Intimacy', 'Scope', 'significance']

In [354]:
def get_summary(data, formula):
    model = smf.ols(formula=formula, data = data)
    result = model.fit()
    summary = result.summary2()
    a = summary.tables[1]
    a['significance'] = a['P>|t|'].apply(lambda x: star_pvals(x))
#     a.sort_values(by=['P>|t|'], inplace=True)
    a['sig'] = a['P>|t|'].apply(lambda x: no_star_pvals(x))
    a['abs_coef'] = a['Coef.'].abs()
    a.sort_values(['sig','abs_coef'], ascending=[False, False], inplace=True)
    a.drop(columns=['sig', 'abs_coef'], inplace=True)

    return summary

In [353]:
def get_summary_r(data, formula):
    model = smf.ols(formula=formula, data = data)
    result = model.fit_regularized(alpha=0.001, L1_wt=0)
    model, result = convert_regularized_to_ols(model, result)
    summary = result.summary2()
    a = summary.tables[1]
    a['significance'] = a['P>|t|'].apply(lambda x: star_pvals(x))
    a['sig'] = a['P>|t|'].apply(lambda x: no_star_pvals(x))
    a['abs_coef'] = a['Coef.'].abs()
    a.sort_values(['sig','abs_coef'], ascending=[False, False], inplace=True)
    a.drop(columns=['sig', 'abs_coef'], inplace=True)
#     a.sort_values(by=['P>|t|'], inplace=True)
    return summary

In [55]:
def get_model(data, formula):
    model = smf.ols(formula=formula, data = data).fit()

    return model

In [344]:
def generate_summary_file_r(data, formula, filename):
    t = get_summary_r(data, formula).as_text()

    with open('Linear Regression/Relative Week/summary/'+filename+'.txt', 'w') as the_file:
        the_file.write(t)

In [343]:
def generate_summary_file(data, formula, filename):
    t = get_summary(data, formula).as_text()

    with open('Linear Regression/Relative Week/summary/'+filename+'.txt', 'w') as the_file:
        the_file.write(t)

In [348]:
def generate_summary_files(dependent_variable):
    data = build_merged_df(dependent_variable)
    all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
    all_data = all_data[all_data[dependent_variable].notnull()]

    generate_summary_file(all_data, dependent_variable+'~'+' + '.join(control_variables), 'all_demo_trait_'+dependent_variable)
    generate_summary_file(all_data, dependent_variable+'~'+' + '.join(sm_life_event_variables+sr_life_event_variables), 'all_sm_sr_'+dependent_variable)
    generate_summary_file(all_data, dependent_variable+'~'+' + '.join(control_variables+sr_life_event_variables+sm_life_event_variables), 'all_sm_sr_demo_trait_'+dependent_variable)
    generate_summary_file_r(all_data, dependent_variable+'~'+' + '.join(control_variables), 'all_demo_trait_'+dependent_variable+'_regularized')
    generate_summary_file_r(all_data, dependent_variable+'~'+' + '.join(sm_life_event_variables+sr_life_event_variables), 'all_sm_sr_'+dependent_variable+'_regularized')
    generate_summary_file_r(all_data, dependent_variable+'~'+' + '.join(control_variables+sr_life_event_variables+sm_life_event_variables), 'all_sm_sr_demo_trait_'+dependent_variable+'_regularized')
    all_variables = ' + '.join(control_variables+sr_life_event_variables+sm_life_event_variables)
    combined_control = ' + '.join(control_variables)
    combined_life_event = ' + '.join(sr_life_event_variables+sm_life_event_variables)
    final_var = '('+all_variables+')**2 - ' + ' ('+combined_control+')**2 - ' + ' ('+combined_life_event+')**2' + ' + ' + all_variables

    generate_summary_file(all_data, ModelDesc.from_formula(dependent_variable+'~'+final_var).describe(), 'all_sm_sr_demo_trait_interaction_'+dependent_variable)
    generate_summary_file_r(all_data, ModelDesc.from_formula(dependent_variable+'~'+final_var).describe(), 'all_sm_sr_demo_trait_interaction_'+dependent_variable+'_regularized')

#     data = build_stacked_df(dependent_variable)
#     generate_summary_file(data, dependent_variable+'~'+' + '.join(control_variables), 'stacked_demo_trait_'+dependent_variable)
#     generate_summary_file(data, dependent_variable+'~'+' + '.join(life_event_variables), 'stacked_life_events_'+dependent_variable)
#     generate_summary_file(data, dependent_variable+'~'+' + '.join(control_variables+life_event_variables + ['type']), 'stacked_demo_trait_life_events_'+dependent_variable)
#     all_variables = ' + '.join(control_variables + life_event_variables + ['type'])
#     combined_control = ' + '.join(control_variables+['type'])
#     combined_life_event = ' + '.join(life_event_variables)
#     final_var = '('+all_variables+')**2 - ' + ' ('+combined_control+')**2 - ' + ' ('+combined_life_event+')**2' + ' + ' + all_variables
    
#     generate_summary_file(data, ModelDesc.from_formula(dependent_variable+'~'+final_var).describe(), 'stacked_demo_trait_life_events_interaction_'+dependent_variable)
    

In [355]:
a = ' + '.join(control_variables)
b = ' + '.join(sr_life_event_variables)
c = ' + '.join(sm_life_event_variables)

final_var = ' (' + a + ' + ' + c +')**2' + ' - ('+a+')**2' + ' + ' + a + '+' + b

data = build_merged_df('stress')
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['stress'].notnull()]

generate_summary_file(all_data, ModelDesc.from_formula('stress'+'~'+final_var).describe(), 'all_stress_life_event_interaction')
generate_summary_file_r(all_data, ModelDesc.from_formula('stress'+'~'+final_var).describe(), 'all_stress_life_event_interaction_regularized')




/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


In [1217]:
a = ' + '.join(control_variables)
b = ' + '.join(sr_life_event_variables)
c = ' + '.join(sm_life_event_variables)

final_var = a + ' + (' + b + ')**2 + ' + '+ (' + c + ')**2'

data = build_merged_df('sleep')
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['sleep'].notnull()]

generate_summary_file(all_data, ModelDesc.from_formula('sleep'+'~'+final_var).describe(), 'all_sleep_sm_sr_interaction')



In [349]:
generate_summary_files('stress')

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


In [350]:
generate_summary_files('sleep')

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


In [351]:
generate_summary_files('anxiety')

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


In [171]:
def convert_regularized_to_ols(model, results):
    pinv_wexog,_ = pinv_extended(model.wexog)
    normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))

    results = sm.regression.linear_model.OLSResults(model, results.params, normalized_cov_params)
    return model, results

In [253]:
def run_regression_regularized(data, dependent_variable, variables):

    control_data = data[variables['control']+[dependent_variable]].dropna()
    train_data, test_data = train_test_split(control_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values

    baseline_formula = dependent_variable+'~'+' + '.join(variables['control'])
    baseline_model = smf.ols(formula=baseline_formula,
                  data = control_data)
    baseline = baseline_model.fit_regularized(alpha=0.001, L1_wt=0.5)
    baseline_model, baseline = convert_regularized_to_ols(baseline_model, baseline)
    
    baseline_train = smf.ols(formula=baseline_formula,
                  data = train_data).fit_regularized()
    f_obs = baseline_train.predict(test_data).values
    pearson_baseline = scipy.stats.pearsonr(f_obs, f_exp)
    
    
    life_events_data = data[variables['life_events']+[dependent_variable]].dropna()    
    train_data, test_data = train_test_split(life_events_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values

    life_events_formula = dependent_variable+'~'+' + '.join(variables['life_events'])
    life_events_model = smf.ols(formula=life_events_formula, data = life_events_data)
    life_events = life_events_model.fit_regularized(alpha=0.001, L1_wt=0.5) 
    life_events_model, life_events = convert_regularized_to_ols(life_events_model, life_events)
    life_events_train = smf.ols(formula=life_events_formula, data = train_data).fit_regularized()     
    f_obs = life_events_train.predict(test_data).values
    pearson_life_events = scipy.stats.pearsonr(f_obs, f_exp)    

    
    all_variable_data = data[variables['all_variables']+[dependent_variable]].dropna()
    train_data, test_data = train_test_split(all_variable_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values    
    
    all_variables_formula = dependent_variable+'~'+' + '.join(variables['all_variables'])
    all_variables_model = smf.ols(formula=all_variables_formula, data = all_variable_data)
    all_variables = all_variables_model.fit_regularized(alpha=0.001, L1_wt=0.5) 
    all_variables_model, all_variables = convert_regularized_to_ols(all_variables_model, all_variables)

    all_variables_train = smf.ols(formula=all_variables_formula, data = train_data).fit_regularized() 
    f_obs = all_variables_train.predict(test_data).values
    pearson_all_variables = scipy.stats.pearsonr(f_obs, f_exp)    

    
    print("Baseline R-square adj", baseline.rsquared_adj)
    print("Life Events R-square adj", life_events.rsquared_adj)
    print("All Variables R-square adj", all_variables.rsquared_adj)
    print()
    print(anova_lm(baseline, life_events))
    print(anova_lm(baseline, all_variables))
    print(anova_lm(life_events, all_variables))
    print()
    print("Baseline pearson", pearson_baseline)
    print("Life Events pearson", pearson_life_events)
    print("All Variables pearson", pearson_all_variables)    
    return all_variables.rsquared_adj

In [160]:
def run_regression(data, dependent_variable, variables):

    control_data = data[variables['control']+[dependent_variable]].dropna()
    train_data, test_data = train_test_split(control_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values

    baseline_formula = dependent_variable+'~'+' + '.join(variables['control'])
    baseline = smf.ols(formula=baseline_formula,
                  data = control_data).fit()
    baseline_train = smf.ols(formula=baseline_formula,
                  data = train_data).fit()
    f_obs = baseline_train.predict(test_data).values
    pearson_baseline = scipy.stats.pearsonr(f_obs, f_exp)
    
    
    life_events_data = data[variables['life_events']+[dependent_variable]].dropna()    
    train_data, test_data = train_test_split(life_events_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values

    life_events_formula = dependent_variable+'~'+' + '.join(variables['life_events'])
    life_events = smf.ols(formula=life_events_formula, data = life_events_data).fit() 
    life_events_train = smf.ols(formula=life_events_formula, data = train_data).fit()     
    f_obs = life_events_train.predict(test_data).values
    pearson_life_events = scipy.stats.pearsonr(f_obs, f_exp)    

    
    all_variable_data = data[variables['all_variables']+[dependent_variable]].dropna()
    train_data, test_data = train_test_split(all_variable_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values    
    
    all_variables_formula = dependent_variable+'~'+' + '.join(variables['all_variables'])
    all_variables = smf.ols(formula=all_variables_formula, data = all_variable_data).fit() 
    all_variables_train = smf.ols(formula=all_variables_formula, data = train_data).fit() 
    f_obs = all_variables_train.predict(test_data).values
    pearson_all_variables = scipy.stats.pearsonr(f_obs, f_exp)    

    print("Baseline R-square adj", baseline.rsquared_adj)
    print("Life Events R-square adj", life_events.rsquared_adj)
    print("All Variables R-square adj", all_variables.rsquared_adj)
    print()
    print(anova_lm(baseline, life_events))
    print(anova_lm(baseline, all_variables))
    print(anova_lm(life_events, all_variables))
    print()
    print("Baseline pearson", pearson_baseline)
    print("Life Events pearson", pearson_life_events)
    print("All Variables pearson", pearson_all_variables)    
    return all_variables.rsquared_adj

In [249]:
def run_all_regression(dependent_variable, is_regularized=False):
    data = build_merged_df(dependent_variable)

    social_media_data = data[data['LifeEventType_sm'].notnull()]
    social_media_data = social_media_data[social_media_data[dependent_variable].notnull()]

    social_media_variables = {
        'control': control_variables,
        'life_events': sm_life_event_variables,
        'all_variables': sm_life_event_variables + control_variables
    }

    survey_data = data[data['LifeEventType_sr'].notnull()]
    survey_data = survey_data[survey_data[dependent_variable].notnull()]
    survey_variables = {
        'control': control_variables,
        'life_events': sr_life_event_variables,
        'all_variables': sr_life_event_variables + control_variables
    }
    
    all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
    all_data = all_data[all_data[dependent_variable].notnull()]
    all_data_variables = {
        'control': control_variables,
        'life_events': sm_life_event_variables + sr_life_event_variables,
        'all_variables': sm_life_event_variables + sr_life_event_variables + control_variables
    }
    
    
#     print("---------SOCIAL MEDIA---------")
#     if is_regularized:
#         run_regression_regularized(social_media_data, dependent_variable, social_media_variables)
#     else:
#         run_regression(social_media_data, dependent_variable, social_media_variables)
#     print()
    
#     print("---------SURVEY---------")
#     if is_regularized:
#         run_regression_regularized(survey_data, dependent_variable, survey_variables)
#     else:
#         run_regression(survey_data, dependent_variable, survey_variables)
#     print()
    
    print("---------SOCIAL MEDIA + SURVEY---------")
    if is_regularized:
        run_regression_regularized(all_data, dependent_variable, all_data_variables) 
    else:
        run_regression(all_data, dependent_variable, all_data_variables)
    print()    

In [60]:
def run_stacked_regression(dependent_variable):
    data = build_stacked_df(dependent_variable)

    all_data = data[data[dependent_variable].notnull()]
    all_data_variables = {
        'control': control_variables,
        'life_events': life_event_variables,
        'all_variables': life_event_variables + control_variables + ['type']
    }
    
    print("---------SOCIAL MEDIA + SURVEY---------")
    run_regression(all_data, dependent_variable, all_data_variables)
    print()    

### Stress 

In [254]:
run_all_regression('stress', True)

---------SOCIAL MEDIA + SURVEY---------
Baseline R-square adj 0.16422305748023447
Life Events R-square adj 0.3179276386084303
All Variables R-square adj 0.3430250035317137

   df_resid        ssr  df_diff    ss_diff         F    Pr(>F)
0     131.0  81.553530      0.0        NaN       NaN       NaN
1     125.0  63.506993      6.0  18.046537  5.920128  0.000018
   df_resid        ssr  df_diff    ss_diff         F    Pr(>F)
0     131.0  81.553530      0.0        NaN       NaN       NaN
1     109.0  53.340419     22.0  28.213111  2.620586  0.000534
   df_resid        ssr  df_diff    ss_diff         F    Pr(>F)
0     125.0  63.506993      0.0        NaN       NaN       NaN
1     109.0  53.340419     16.0  10.166574  1.298448  0.211203

Baseline pearson (0.31611918522334476, 0.08878555471555973)
Life Events pearson (0.41344540003192787, 0.02314694853629488)
All Variables pearson (0.44591655005369873, 0.013520427457533346)



In [242]:
run_all_regression('stress', False)

---------SOCIAL MEDIA---------
Baseline R-square adj 0.1008882646701027
Life Events R-square adj 0.06087553832060044
All Variables R-square adj 0.1555923531285176

   df_resid         ssr  df_diff   ss_diff         F  Pr(>F)
0     615.0  252.777309      0.0       NaN       NaN     NaN
1     619.0  265.743779     -4.0 -12.96647  7.550736     NaN
   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0     615.0  252.777309      0.0        NaN       NaN       NaN
1     603.0  232.765588     12.0  20.011721  4.320179  0.000001
   df_resid         ssr  df_diff    ss_diff         F        Pr(>F)
0     619.0  265.743779      0.0        NaN       NaN           NaN
1     603.0  232.765588     16.0  32.978191  5.339559  1.187143e-10

Baseline pearson (0.3269397950664338, 0.00017568539380028838)
Life Events pearson (0.22618431090705632, 0.01055815059769284)
All Variables pearson (0.3926633185717482, 4.968582704067654e-06)

---------SURVEY---------
Baseline R-square adj 0.192469183783334

In [141]:
run_stacked_regression('stress')

---------SOCIAL MEDIA + SURVEY---------
Baseline R-square adj 0.1764291746827119
Life Events R-square adj 0.037265771142125925
All Variables R-square adj 0.20533117142664092

   df_resid          ssr  df_diff     ss_diff           F  Pr(>F)
0    7296.0  2855.129332      0.0         NaN         NaN     NaN
1    7299.0  3338.948984     -3.0 -483.819652  352.546031     NaN
   df_resid          ssr  df_diff     ss_diff          F        Pr(>F)
0    7296.0  2855.129332      0.0         NaN        NaN           NaN
1    7282.0  2749.646473     14.0  105.482859  19.953895  2.572230e-50
   df_resid          ssr  df_diff     ss_diff          F         Pr(>F)
0    7299.0  3338.948984      0.0         NaN        NaN            NaN
1    7282.0  2749.646473     17.0  589.302511  91.804334  7.315350e-291

Baseline pearson (0.43201629980000694, 1.3966803892644736e-67)
Life Events pearson (0.21195019763458153, 2.5426632972018607e-16)
All Variables pearson (0.455476884757731, 7.856886523692654e-76)



In [255]:
run_all_regression('sleep', True)

---------SOCIAL MEDIA + SURVEY---------
Baseline R-square adj 0.31273109715001657
Life Events R-square adj 0.1847110446316672
All Variables R-square adj 0.3686259005736009

   df_resid         ssr  df_diff    ss_diff         F  Pr(>F)
0     131.0  201.950365      0.0        NaN       NaN     NaN
1     125.0  228.595789      6.0 -26.645424 -2.428361     1.0
   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0     131.0  201.950365      0.0        NaN       NaN       NaN
1     109.0  154.368937     22.0  47.581428  1.527149  0.079689
   df_resid         ssr  df_diff    ss_diff         F   Pr(>F)
0     125.0  228.595789      0.0        NaN       NaN      NaN
1     109.0  154.368937     16.0  74.226852  3.275727  0.00012

Baseline pearson (0.5769699009377611, 0.0008444798674519633)
Life Events pearson (0.4255037779923419, 0.01906743312753837)
All Variables pearson (0.50393478307934, 0.004521865835320618)



In [243]:
run_all_regression('sleep', False)

---------SOCIAL MEDIA---------
Baseline R-square adj 0.12888423999983512
Life Events R-square adj 0.03501048666045281
All Variables R-square adj 0.1460516677563436

   df_resid          ssr  df_diff     ss_diff          F  Pr(>F)
0     615.0   986.807101      0.0         NaN        NaN     NaN
1     619.0  1100.257968     -4.0 -113.450866  15.956732     NaN
   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0     615.0  986.807101      0.0        NaN       NaN       NaN
1     603.0  948.484387     12.0  38.322715  2.030309  0.019829
   df_resid          ssr  df_diff     ss_diff         F        Pr(>F)
0     619.0  1100.257968      0.0         NaN       NaN           NaN
1     603.0   948.484387     16.0  151.773581  6.030639  1.998141e-12

Baseline pearson (0.28058756354422637, 0.0013971575366344795)
Life Events pearson (0.2284233227822866, 0.0097947700864724)
All Variables pearson (0.3168717807213702, 0.00028380387240432636)

---------SURVEY---------
Baseline R-square adj

In [142]:
run_stacked_regression('sleep')

---------SOCIAL MEDIA + SURVEY---------
Baseline R-square adj 0.06471969801113886
Life Events R-square adj 0.025825310498153065
All Variables R-square adj 0.09508662229032705

   df_resid           ssr  df_diff     ss_diff          F  Pr(>F)
0    7296.0  10189.902958      0.0         NaN        NaN     NaN
1    7299.0  10618.022480     -3.0 -428.119522  98.098756     NaN
   df_resid           ssr  df_diff     ss_diff          F        Pr(>F)
0    7296.0  10189.902958      0.0         NaN        NaN           NaN
1    7282.0   9840.136395     14.0  349.766563  18.488421  3.284829e-46
   df_resid           ssr  df_diff     ss_diff          F         Pr(>F)
0    7299.0  10618.022480      0.0         NaN        NaN            NaN
1    7282.0   9840.136395     17.0  777.886085  33.862314  5.700353e-107

Baseline pearson (0.2606047902620883, 3.862609272566685e-24)
Life Events pearson (0.1782973520119464, 6.464970542162705e-12)
All Variables pearson (0.3215680962420773, 1.5169420758665593e-36

In [256]:
run_all_regression('anxiety', True)

---------SOCIAL MEDIA + SURVEY---------
Baseline R-square adj 0.24627583377637408
Life Events R-square adj 0.25932666364816437
All Variables R-square adj 0.3349336128389523

   df_resid        ssr  df_diff   ss_diff         F    Pr(>F)
0     131.0  54.675338      0.0       NaN       NaN       NaN
1     125.0  51.267774      6.0  3.407564  1.384708  0.225832
   df_resid        ssr  df_diff    ss_diff         F   Pr(>F)
0     131.0  54.675338      0.0        NaN       NaN      NaN
1     109.0  40.142021     22.0  14.533317  1.793781  0.02591
   df_resid        ssr  df_diff    ss_diff         F    Pr(>F)
0     125.0  51.267774      0.0        NaN       NaN       NaN
1     109.0  40.142021     16.0  11.125753  1.888151  0.028937

Baseline pearson (0.4583087966953588, 0.010864283357161187)
Life Events pearson (0.4327004317257537, 0.016929535687524738)
All Variables pearson (0.5093966370918712, 0.00403861048919734)



In [246]:
run_all_regression('anxiety', False)

---------SOCIAL MEDIA---------
Baseline R-square adj 0.16702073399945816
Life Events R-square adj 0.058810636910408776
All Variables R-square adj 0.20562051777629053

   df_resid         ssr  df_diff    ss_diff          F  Pr(>F)
0     615.0  175.905995      0.0        NaN        NaN     NaN
1     619.0  200.050201     -4.0 -24.144206  18.676892     NaN
   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0     615.0  175.905995      0.0        NaN       NaN       NaN
1     603.0  164.481351     12.0  11.424644  3.490295  0.000052
   df_resid         ssr  df_diff    ss_diff         F        Pr(>F)
0     619.0  200.050201      0.0        NaN       NaN           NaN
1     603.0  164.481351     16.0  35.568851  8.149867  7.180352e-18

Baseline pearson (0.3455393549529829, 6.926441774403982e-05)
Life Events pearson (0.10890120526473099, 0.22293974870293615)
All Variables pearson (0.36324173515787683, 2.7006480237832376e-05)

---------SURVEY---------
Baseline R-square adj 0.25919

In [152]:
run_stacked_regression('anxiety')

---------SOCIAL MEDIA + SURVEY---------
Baseline R-square adj 0.24437320723951328
Life Events R-square adj 0.06283115238978265
All Variables R-square adj 0.2745631707396997

   df_resid          ssr  df_diff     ss_diff           F  Pr(>F)
0    7296.0  2427.075649      0.0         NaN         NaN     NaN
1    7299.0  3011.427046     -3.0 -584.351397  472.110706     NaN
   df_resid          ssr  df_diff     ss_diff          F        Pr(>F)
0    7296.0  2427.075649      0.0         NaN        NaN           NaN
1    7282.0  2325.634246     14.0  101.441404  22.688014  5.557028e-58
   df_resid          ssr  df_diff   ss_diff           F  Pr(>F)
0    7299.0  3011.427046      0.0       NaN         NaN     NaN
1    7282.0  2325.634246     17.0  685.7928  126.314516     0.0

Baseline pearson (0.5110938369211977, 3.646142742973813e-98)
Life Events pearson (0.2677242857743038, 1.961243299780616e-25)
All Variables pearson (0.536659938372349, 6.668266538579121e-110)



In [257]:
def stress_level(row):
    if row['stress'] >= 4.0:
        return 'High'
    elif row['stress'] <= 2.0:
        return 'Low'
    else:
        return 'Medium'
    
def sleep_level(row):
    if row['sleep'] >= 8.0:
        return 'High'
    elif row['sleep'] <= 4.0:
        return 'Low'
    else:
        return 'Medium'
    
def anxiety_level(row):
    if row['anxiety'] >= 3.0:
        return 'High'
    elif row['anxiety'] <= 2.0:
        return 'Low'
    else:
        return 'Medium'

In [280]:
df_social_media_data = pd.read_csv('data/Superimposed/Facebook Data For Life Events-Combined - FB Data.csv')
df_social_media_data = df_social_media_data[['snapshot_id', 'created_time', 'Text', 'final_life_event_category_2', 'ended/ongoing']]
df_social_media_data = df_social_media_data.replace({'PostiveMove':'PositiveMove', 'Negative Move':'NegativeMove'})
df_social_media_data = df_social_media_data.drop(df_social_media_data[((df_social_media_data['final_life_event_category_2'].isnull() == True))].index)
df_social_media_data['created_date'] = pd.to_datetime(df_social_media_data['created_time'], format = '%Y-%m-%d %H:%M:%S').dt.date
df_social_media_data['valence'] = df_social_media_data.apply(compute_sentiment, axis=1)
df_social_media_data['day'] = df_social_media_data['created_date']


In [281]:
df_social_media_data[df_social_media_data['Text'] == 'Dinner on the airplane, Bibimbap! Oh and by the way....we arrived to Seoul!']



,snapshot_id,created_time,Text,final_life_event_category_2,ended/ongoing,created_date,valence,day
1912,12840161237957324034,2018-03-17 10:23:39,"Dinner on the airplane, Bibimbap! Oh and by th...",Vacation,ongoing,2018-03-17,0,2018-03-17


In [287]:
df_dailies = calculate_avg_mental_health()
df_demographics = load_demographics_data()
median_per_individual = df_dailies.groupby(['snapshot_id']).median().reset_index()[['snapshot_id','stress', 'sleep', 'anxiety']]

merged_data = pd.merge(df_dailies, df_demographics, how="inner", on="snapshot_id")
merged_data_sm = pd.merge(merged_data, df_social_media_data, how="right", on=["snapshot_id", "day"])


merged_data_sm['stress_imputed'] = False
merged_data_sm['sleep_imputed'] = False
merged_data_sm['anxiety_imputed'] = False

for i, row in merged_data_sm.iterrows():
    if row['stress'] != row['stress']:
        merged_data_sm._set_value(i,'stress',median_per_individual[median_per_individual['snapshot_id'] == row['snapshot_id']]['stress'].values[0])
        merged_data_sm._set_value(i,'stress_imputed', True)
    if row['sleep'] != row['sleep']:
        merged_data_sm._set_value(i,'sleep',median_per_individual[median_per_individual['snapshot_id'] == row['snapshot_id']]['sleep'].values[0])
        merged_data_sm._set_value(i,'sleep_imputed', True)
    if row['anxiety'] != row['anxiety']:
        merged_data_sm._set_value(i,'anxiety',median_per_individual[median_per_individual['snapshot_id'] == row['snapshot_id']]['anxiety'].values[0])
        merged_data_sm._set_value(i,'anxiety_imputed', True)


merged_data_sm = merged_data_sm[['snapshot_id', 'Text', 'day', 'stress', 'sleep', 'anxiety', 'stress_imputed', 'sleep_imputed', 'anxiety_imputed']]
# merged_data_sm = merged_data_sm[['snapshot_id', 'Text', 'stress', 'sleep', 'anxiety', 'day']]
merged_data_sm = merged_data_sm.sort_values(['snapshot_id', 'day'])
merged_data_sm.to_csv('Linear Regression/Relative Week/social_media_post_mental_health_labels.csv')
merged_data_sm


,snapshot_id,Text,day,stress,sleep,anxiety,stress_imputed,sleep_imputed,anxiety_imputed
1890,9251084965720942673,This guy will be missed. Will always wish we ...,2018-10-07,2.666667,9.000000,2.166667,True,True,True
1131,9252537403715547191,Family reunion,2018-06-10,1.600000,7.500000,1.000000,False,False,False
1132,9252537403715547191,"Oh Zurich, you are so beautiful. I should visi...",2018-07-01,1.333333,7.000000,1.400000,False,False,False
1133,9252537403715547191,From Poland with love! Happy 4th of July to al...,2018-07-04,1.800000,7.666667,1.366667,True,True,True
1134,9252537403715547191,Spontaneously available for activities in DC d...,2018-08-03,1.800000,7.500000,1.400000,False,False,False
...,...,...,...,...,...,...,...,...,...
506,13713449702891591320,What a beautiful weekend we had celebrating ou...,2018-08-06,2.000000,5.000000,1.500000,True,True,True
583,13721933279774980118,#chicagothistime #friends #vacationmode #chica...,2018-07-01,2.250000,7.000000,2.250000,False,False,False
1027,13725566228858842118,"Thank you, Aspen for the fresh snow and great ...",2018-12-02,1.333333,9.000000,1.000000,True,True,True
151,13804268872491564932,"Ate, drank, and surfed our way through Central...",2018-11-23,1.400000,7.000000,1.750000,True,True,True
